In [ ]:
#Multiple Kernel ICA
import numpy as np
import matplotlib.pyplot as plt
import librosa
import soundfile as sf
from sklearn.decomposition import FastICA
from sklearn.kernel_approximation import RBFSampler
from sklearn.model_selection import train_test_split
from IPython.display import Audio, display

In [ ]:
# Load audio files
audio_files = [
    '/content/mixed1.wav',
    '/content/mixed2.wav'
]

# Load audio signals
signals = []
sampling_rates = []

for audio_file in audio_files:
    sig, sr = librosa.load(audio_file, sr=None)
    signals.append(sig)
    sampling_rates.append(sr)

# Choosing the minimum length of the signals
min_length = min(len(sig) for sig in signal)#finding the minimum length out of all audio signals in the list

# Trimming signals to have the same length
signals = [sig[:min_length] for sig in signal]#Here we are trimming signals to have same length over all audio signals

# Mixing Signals
X = np.column_stack(signals)


In [ ]:
sf.write(f'mixed_signal2.wav', X, sampling_rates[0])

In [ ]:
from IPython.display import Audio, display
print('mixed1')
display(Audio('/content/mixed1.wav',rate=sampling_rates[0]))
print('mixed2')
display(Audio('/content/mixed2.wav',rate=sampling_rates[0]))
print('mixed signal')
display(Audio('/content/mixed_signal2.wav',rate=sampling_rates[0]))

In [ ]:
# Creating multiple kernel representations using RBFSampler
n_kernels = 15
rbf_samplers = [RBFSampler(n_components=1000, random_state=i).fit(X) for i in range(n_kernels)] #matrix=X
kernel_representations = [rbf.transform(X) for rbf in rbf_samplers]  #matrix=X

# Reshape for MKL input
X_reshaped = np.array(kernel_representations).reshape(len(kernel_representations), -1)

# Perform MKL with ICA manually
weights = [1.0 / n_kernels] * n_kernels  # Equal weights for simplicity
combined_kernel = sum(weight * kernel for weight, kernel in zip(weights, kernel_representations))

# Fitting ICA on the combined kernel
ica_mkl = FastICA(n_components=len(audio_files))
S_mkl = ica_mkl.fit_transform(combined_kernel)
plt.title(f'Mixed Signal')
plt.plot(X)
plt.title(f'Mixed Signal for MKL-ICA')
plt.plot(combined_kernel)

In [ ]:
# Plotting the original and estimated sources
plt.figure(figsize=(15, 8))
for i in range(len(audio_files)):
    plt.subplot(3, len(audio_files), len(audio_files) + i + 1)
    plt.title(f'MKL-ICA Estimated Source {i + 1}')
    plt.plot(S_mkl[:, i])

plt.tight_layout()
plt.show()


# Save the estimated signals to audio files
for i in range(len(audio_files)):
    sf.write(f'estimated_signal2_mkl_ica_{i + 1}.wav', S_mkl[:, i], sampling_rates[i])
    print("Estimated signal: ",i+1)
    display(Audio(S_mkl[:,i],rate=sampling_rates[0]))
